In [ ]:
import tensorflow as tf
import numpy as np
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline

# 加载数据

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

data_dir='../../data/2_class'
data_root=pathlib.Path(data_dir)

In [ ]:
for item in data_root.iterdir():
    print(item)

In [ ]:
#所有数据地址
all_image_path=list(data_root.glob('*/*'))
len(all_image_path)

In [ ]:
all_image_path[:3],all_image_path[-3:]

**图片地址**

In [ ]:
all_image_path=[str(path)for path in all_image_path]
all_image_path[0:2]

In [ ]:
import random

In [ ]:
random.shuffle(all_image_path)#乱序
all_image_path[0:2]

In [ ]:
image_count=len(all_image_path)
image_count

**通过sorted排列列表**

In [ ]:
#将label返回成列表
labels_names=sorted(item.name for item in data_root.glob('*/'))
labels_names

**enumerate方法将可迭代对象转化为元组列表**

In [ ]:
#编码
label_to_index=dict((name,index) for index,name in enumerate(labels_names))#enumerate将一个列表转化为索引列表，丛零开始
label_to_index

**通过索引对label进行0/1编码**

In [ ]:
#通过字典，将lake，airplane编码成0和1
all_image_label=[label_to_index[pathlib.Path(p).parent.name] for p in all_image_path]
all_image_label[:5],all_image_path[:5]

**显示图片**

In [ ]:
import IPython.display as display

In [ ]:
index_to_label = dict((v,k) for k,v,in label_to_index.items())
index_to_label

In [ ]:
#查看图像内容
for n in range(1):
    image_index = random.choice(range(len(all_image_path)))
    display.display(display.Image(all_image_path[image_index]))
    print(index_to_label[all_image_label[image_index]])
    print()

# 单张图片的处理过程

In [ ]:
img_path=all_image_path[0]
img_path

In [ ]:
img_raw=tf.io.read_file(img_path)#读取图片
img_tensor=tf.image.decode_image(img_raw)#图片解码

In [ ]:
img_tensor=tf.cast(img_tensor,tf.float32)#转换数据类型
img_tensor=img_tensor/255#图片归一化
print(img_tensor.shape)

In [ ]:
def load_preprocess_image(path):
    img_raw=tf.io.read_file(path)#读取图片
    img_tensor=tf.image.decode_jpeg(img_raw,channels=3)#图片解码
    img_tensor=tf.image.resize(img_tensor,[256,256])
    img_tensor=tf.cast(img_tensor,tf.float32)#转换数据类型
    img=img_tensor/255#图片归一化
    return img

In [ ]:
#显示图片
image_path=all_image_path[20]
plt.imshow(load_preprocess_image(image_path))

# 构造tf.data

In [ ]:
path_ds=tf.data.Dataset.from_tensor_slices(all_image_path)#从目录加载数据
AUTOTUNE=tf.data.experimental.AUTOTUNE
image_dateset=path_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
label_dataset=tf.data.Dataset.from_tensor_slices(tf.cast(all_image_label,tf.int64))#这里加载时，通过cast方法将数据转化为64格式

In [ ]:
image_dateset,label_dataset

In [ ]:
for img in image_dateset.take(1):
    plt.imshow(img)

tf.data.Dataset.zip() 方法可以用於合併兩個數據集，以便可以一起處理它們。
例如，如果 dataset1 包含圖像數據，dataset2 包含標籤數據，那麼 tf.data.Dataset.zip(dataset1, dataset2) 會創建一個數據集，
其中包含圖像和標籤的元組。然後，這個數據集可以一起用於訓練一個圖像分類器。

In [ ]:
dataset=tf.data.Dataset.zip((image_dateset,label_dataset))

In [ ]:
dataset

In [ ]:
test_count=int(image_count*0.2)
train_count=image_count-test_count

In [ ]:
test_count,train_count

In [ ]:
#创建训练数据以及测试数据
train_data = dataset.skip(test_count)
test_data=dataset.take(test_count)

In [ ]:
BATCH_SIZE=32

**tf.data.experimental.shuffle_and_repeat() 方法是一種用於將數據集打亂並重複**
apply方法作用是对数据进行处理，将数据转化为另一个数据

tf.data.Dataset.prefetch() 方法會在數據集上預取 buffer_size 個數據。
這意味著當您從數據集中請求數據時，數據集將已經預取了 buffer_size 個數據。這可以提高數據集的性能，因為它可以減少數據集從磁碟或網絡流中讀取數據的次數。

In [ ]:
# train_data=train_dataset.repeat().shuffle(buffer_size=train_count).batch(BATCH_SIZE)
train_data = train_data.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=train_count))
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)#预加载数据

test_data=test_data.batch(BATCH_SIZE)

In [ ]:
train_data,test_data

# 构建模型

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(512,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D())#默认2x2
# model.add(tf.keras.layers.Conv2D(1024,(3,3),activation='relu'))
# model.add(tf.keras.layers.GlobalAveragePooling2D())#全局平均池化
# model.add(tf.keras.layers.Dense(1024,activation='relu'))
# model.add(tf.keras.layers.Dense(256,activation='relu'))
# # model.add(tf.keras.layers.Dense(2,activation='sigmoid'))
# model.add(tf.keras.layers.Dense(1,activation='sigmoid'))


## 增加BN层

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3)))#卷积层
model.add(tf.keras.layers.BatchNormalization())#批标准化层
model.add(tf.keras.layers.Activation('relu'))#激活层

model.add(tf.keras.layers.Conv2D(64,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPooling2D())#连接层
model.add(tf.keras.layers.Conv2D(128,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(128,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPooling2D())#连接层
model.add(tf.keras.layers.Conv2D(256,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(256,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Conv2D(512,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(1024,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['acc']
# )

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc']
)

In [ ]:
steps_per_epoch=train_count//BATCH_SIZE
validation_steps=test_count//BATCH_SIZE

In [ ]:
import os,datetime
log_dir = os.path.join(
    'logs', datetime.datetime.now().strftime("%Y%m%d-%H:%M:%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(train_data,
                    epochs=5,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_data,
                    validation_steps=validation_steps,
                    callbacks=[tensorboard_callback])

In [ ]:
plt.plot(history.epoch,history.history.get('acc'),label='acc')
plt.plot(history.epoch,history.history.get('val_acc'),label='val_acc')